# 7. GUI creation

Two options: <br/>
* [pyqt and pyqtgraph](#pyqt_and_pyqtgraph) <br/>
* [napari](#napari)

In [1]:
import os
import time
from useq import MDAEvent, MDASequence, Position
from pymmcore_plus.mda import mda_listeners_connected
from pymmcore_plus.mda.handlers import ImageSequenceWriter
import matplotlib.pyplot as plt
import matplotlib.image as img
import cv2
import numpy as np
from scipy.optimize import curve_fit
import glob
import pandas as pd
import m2stitch
from os import path

In [2]:
# set micro manager directory
mm_dir = 'D:\ProgramFiles\Micro-Manager-2.0'

# load micro manager system configuration
import os
from pymmcore_plus import CMMCorePlus, Device, DeviceAdapter, ConfigGroup
core = CMMCorePlus()
#core_plus.setDeviceAdapterSearchPaths([mm_dir])
core.loadSystemConfiguration(os.path.join(mm_dir, 'MMConfig_Edge42_SOLA_ASIStage_PixelSize.cfg'))

## pyqt and pyqtgraph <a id='pyqt_and_pyqtgraph'></a>

In [3]:
from PyQt5.QtCore import QTimer
from PyQt5.QtWidgets import QApplication, QLabel, QPushButton, QWidget, QGridLayout
from PyQt5.QtGui import QFont
import pyqtgraph as pg

In [3]:
### simple plot

# start application
app = pg.mkQApp()

# create window
plot_widget = pg.PlotWidget()
x = np.linspace(0, 20, 1000)
y = np.sin(x)
plot_widget.plot(x, y)
plot_widget.show()

# execute application
app.exec()


0

In [ ]:
### moving sine curve ('live data acquisition')

class window(QWidget):
    def __init__(self):
        super().__init__()
        self.t = np.linspace(0, 2*np.pi, 1000)
        self.num = 0.01
        self.graph = pg.PlotWidget(self)
        self.grid = QGridLayout()
        self.timer = QTimer()
        self.grid.addWidget(self.graph, 0, 0, 1, 1)
        self.setLayout(self.grid)
        self.timer.timeout.connect(self.update)
        self.timer.start(1000) # in ms
        self.graph.show()
        self.setGeometry(0, 0, 500, 200)
        self.show()

    def update(self):
        self.graph.plotItem.clear()
        data = np.sin(self.t + self.num)
        self.graph.plotItem.plot(self.t, data)
        self.num = self.num + 0.5

app = QApplication(['auto update'])

win = window()

app.exec_()

In [ ]:
### display live microscopy images

class window(QWidget):
    def __init__(self):
        super().__init__()
        self.graph = pg.PlotWidget(self)
        self.grid = QGridLayout()
        self.timer = QTimer()
        self.grid.addWidget(self.graph)
        self.setLayout(self.grid)
        self.timer.timeout.connect(self.update)
        self.timer.start(1000) # in ms
        self.graph.show()
        self.show()
    
    def update(self):
        core.setConfig('LED_light', 'on')
        self.graph.plotItem.clear()
        core.snapImage()
        im = core.getImage()
        imv = pg.ImageView()
        imv.setImage(im)
        imv.show()
        #imv.close()
        core.setConfig('LED_light', 'off')

app = QApplication(['auto update'])

win = window()

app.exec_()

## napari <a id='napari'></a>